Import EV charging and vehicle licensing data as dataframes

In [0]:
charging_points_df = spark.read.format("delta").load("/delta/EV_charging_points_silver")
vehicle_licenses_df = spark.read.format("delta").load("/delta/vehicle_licensing_silver")

Use Spark SQL to aggregate number_of_chargers by Town

In [0]:
charging_points_df.createOrReplaceTempView("Charging_Points")
grouped_charging_points_df = spark.sql(
    "SELECT Town, SUM(Number_of_chargers) AS No_of_chargers \
    FROM Charging_Points \
    GROUP BY Town"
)

Use Spark SQL to aggregate No_of_licensed_vehicles by Town for only electric vehicles

In [0]:
vehicle_licenses_df.createOrReplaceTempView("Vehicle_Licenses")
grouped_EV_licenses_df = spark.sql(
    "SELECT Town, SUM(No_of_licensed_vehicles) AS No_of_licensed_electric_vehicles \
    FROM Vehicle_Licenses \
    WHERE Type_of_fuel = 'Electric' \
    GROUP BY Town"
)

Perform an inner join between both aggregated dataframes

In [0]:
fact_df = grouped_charging_points_df.join(
            grouped_EV_licenses_df, 
            grouped_charging_points_df["Town"] == grouped_EV_licenses_df["Town"], 
            "inner"
          ).select(
            grouped_charging_points_df["*"], 
            grouped_EV_licenses_df["No_of_licensed_electric_vehicles"]
          )

Import the demographic data as a dataframe and use Spark SQL to refocus it to population densities in South Dublin

In [0]:
demographic_df = spark.read.format("delta").load("/delta/demographic_data_silver")

demographic_df.createOrReplaceTempView("Demographic_Data")
dim_df = spark.sql(
    "SELECT Statistic_label, Electoral_divisions, Value \
    FROM Demographic_Data \
    WHERE LOWER(Electoral_divisions) LIKE '%south dublin' \
    AND Statistic_label = 'Population density (persons per sq km)'"
)

Extract the left-most word from **Electoral_divisions** before delimiter using regular expressions

In [0]:
from pyspark.sql.functions import regexp_extract, col

# Use regular expression to extract the first word based on multiple delimiters
dim_df = dim_df.withColumn("Electoral_divisions", regexp_extract(col("Electoral_divisions"), r"^[^\s-_, ]+", 0))

Pivot **Statistic_label**, taking the SUM of **Value** as its value

In [0]:
from pyspark.sql.functions import sum, round

# Pivot the DataFrame
dim_df = dim_df.groupBy("Electoral_divisions").pivot("Statistic_label").agg(round(sum("Value"), 1))

Rename some columns

In [0]:
# Dictionary of columns to rename and their new names
columns_to_rename = {
    "Electoral_divisions": "Town",
    "Population density (persons per sq km)": "Population_density"
}

# Rename columns
for old_col, new_col in columns_to_rename.items():
    dim_df = dim_df.withColumnRenamed(old_col, new_col)

Write to gold layer as delta tables

In [0]:
from delta.tables import DeltaTable

fact_df.write.format("delta").mode("append").save("/delta/fact_EV_gold")
dim_df.write.format("delta").mode("append").save("/delta/dim_demography_gold")